# Report for ForestQuery into Global Deforestation, 1990 to 2016

ForestQuery is on a mission to combat deforestation around the world and to raise awareness about this topic and its impact on the environment. The data analysis team at ForestQuery has obtained data from the World Bank that includes forest area and total land area by country and year from 1990 to 2016, as well as a table of countries and the regions to which they belong.

The data analysis team has used SQL to bring these tables together and to query them in an effort to find areas of concern as well as areas that present an opportunity to learn from successes.

## Setting Up Database Connection

In [1]:
import psycopg2
fq_conn = psycopg2.connect("dbname=forest_query")

%load_ext sql_magic
%config SQL.conn_name = 'fq_conn'

## Global Situation

1. What was the total forest area (in sq km) of the world in 1990?
2. What was the total forest area (in sq km) of the world in 2016?
3. What was the change (in sq km) in the forest area of the world from 1990 to 2016?
4. What was the percent change in forest area of the world between 1990 and 2016?
5. If you compare the amount of forest area lost between 1990 and 2016, to which country's total area in 2016 is it closest to?

### 1. World Total Forest Area (sq. km) in 1990

In [2]:
%%read_sql
SELECT forest_area_sqkm
FROM forestation
WHERE country_code = 'WLD' AND year = 1990;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,forest_area_sqkm
0,41282694.9


### 2. World Total Forest Area (sq. km) in 2016

In [3]:
%%read_sql
SELECT forest_area_sqkm
FROM forestation
WHERE country_code = 'WLD' AND year = 2016;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,forest_area_sqkm
0,39958245.9


### 3. World Forest Area Change [1990-2016] (sq. km)

In [4]:
%%read_sql
SELECT ROUND((SUM(forest_area_sqkm) FILTER ( WHERE year = 2016 )
           - SUM(forest_area_sqkm) FILTER ( WHERE year = 1990 ))::numeric
        , 2) forest_area_change
FROM forestation
WHERE country_code = 'WLD';

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,forest_area_change
0,-1324449.0


### 4. World Forest Percentage Change [1990-2016]

In [5]:
%%read_sql
SELECT ROUND((100 * (f_2016.forest_percentage - f_1990.forest_percentage)
           / f_1990.forest_percentage)::numeric, 2) AS forest_pct_change
FROM (SELECT country_code, forest_percentage
      FROM forestation
      WHERE year = 1990 AND
            country_code = 'WLD') f_1990
JOIN (SELECT country_code, forest_percentage
      FROM forestation
      WHERE year = 2016 AND
            country_code = 'WLD') f_2016
ON f_1990.country_code = f_2016.country_code;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,forest_pct_change
0,-3.23


### 5. Most Forest Area Lost Comparing to the Entire World (sq. km)

In [6]:
%%read_sql
WITH forest_area_delta AS (
  SELECT SUM(forest_area_sqkm) FILTER ( WHERE year = 2016 )
             - SUM(forest_area_sqkm) FILTER ( WHERE year = 1990 ) AS value
  FROM forestation
  WHERE country_code = 'WLD'
)

SELECT country_code,
       country_name,
       ROUND(total_area_sqkm::numeric, 2) total_area_sqkm
FROM forestation
WHERE year = 2016 AND
      ABS(total_area_sqkm - (SELECT ABS(value) FROM forest_area_delta))
          = (SELECT MIN(ABS(total_area_sqkm - (SELECT ABS(value)
                                               FROM forest_area_delta)))
             FROM forestation);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_code,country_name,total_area_sqkm
0,PER,Peru,1279999.99


## Regional Outlook
1. What was the percent forest of the entire world in 2016? Which region had the HIGHEST percent forest in 2016, and which had the LOWEST?
2. What was the percent forest of the entire world in 1990? Which region had the HIGHEST percent forest in 1990, and which had the LOWEST?
3. Based on the table you created, which regions of the world DECREASED in forest area from 1990 to 2016?

### 6. Forest Percentage of Entire World in 2016

In [7]:
%%read_sql
SELECT country_name region,
       ROUND(forest_percentage::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 2016 AND
      country_code = 'WLD';

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,World,31.38


### 7. Region with Highest Relative Forestation in 2016

In [8]:
%%read_sql
SELECT region,
       ROUND((SUM(forest_area_sqkm)
             / SUM(total_area_sqkm) * 100)::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 2016
GROUP BY region
HAVING SUM(forest_area_sqkm) / SUM(total_area_sqkm)
    = (SELECT MAX(forest_ratio)
       FROM (SELECT SUM(forest_area_sqkm) / SUM(total_area_sqkm) AS forest_ratio
             FROM forestation
             WHERE year = 2016
             GROUP BY region) t);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,Latin America & Caribbean,46.16


### 8. Region with Lowest Relative Forestation in 2016

In [9]:
%%read_sql
SELECT region,
       ROUND((SUM(forest_area_sqkm)
             / SUM(total_area_sqkm) * 100)::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 2016
GROUP BY region
HAVING SUM(forest_area_sqkm) / SUM(total_area_sqkm)
    = (SELECT MIN(forest_ratio)
       FROM (SELECT SUM(forest_area_sqkm) / SUM(total_area_sqkm) AS forest_ratio
             FROM forestation
             WHERE year = 2016
             GROUP BY region) t);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,Middle East & North Africa,2.07


### 9. Forest Percentage of Entire World in 1990

In [10]:
%%read_sql
SELECT country_name region,
       ROUND(forest_percentage::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 1990 AND
      country_code = 'WLD';

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,World,32.42


### 10. Region with Highest Relative Forestation in 1990

In [11]:
%%read_sql
SELECT region,
       ROUND((SUM(forest_area_sqkm)
             / SUM(total_area_sqkm) * 100)::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 1990
GROUP BY region
HAVING SUM(forest_area_sqkm) / SUM(total_area_sqkm)
    = (SELECT MAX(forest_ratio)
       FROM (SELECT SUM(forest_area_sqkm) / SUM(total_area_sqkm) AS forest_ratio
             FROM forestation
             WHERE year = 1990
             GROUP BY region) t);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,Latin America & Caribbean,51.03


### 11. Region with Lowest Relative Forestation in 1990

In [12]:
%%read_sql
SELECT region,
       ROUND((SUM(forest_area_sqkm)
             / SUM(total_area_sqkm) * 100)::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 1990
GROUP BY region
HAVING SUM(forest_area_sqkm) / SUM(total_area_sqkm)
    = (SELECT MIN(forest_ratio)
       FROM (SELECT SUM(forest_area_sqkm) / SUM(total_area_sqkm) AS forest_ratio
             FROM forestation
             WHERE year = 1990
             GROUP BY region) t);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_percentage
0,Middle East & North Africa,1.78


### 12. Percent Forest Area by Region, 1990 & 2016

In [13]:
%%read_sql
SELECT region, forest_pct_1990, forest_pct_2016
FROM (SELECT region,
             ROUND((100 * SUM(forest_area_sqkm) FILTER ( WHERE year = 1990 )
                       / SUM(total_area_sqkm) FILTER ( WHERE year = 1990 ))::numeric
                       , 2) AS forest_pct_1990,
             ROUND((100 * SUM(forest_area_sqkm) FILTER ( WHERE year = 2016 )
                       / SUM(total_area_sqkm) FILTER ( WHERE year = 2016 ))::numeric
                       , 2) AS forest_pct_2016
      FROM forestation
      GROUP BY region) t
ORDER BY (forest_pct_2016 - forest_pct_1990) / forest_pct_1990 DESC;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,region,forest_pct_1990,forest_pct_2016
0,Middle East & North Africa,1.78,2.07
1,South Asia,16.51,17.51
2,East Asia & Pacific,25.78,26.36
3,Europe & Central Asia,37.28,38.04
4,North America,35.65,36.04
5,World,32.42,31.38
6,Sub-Saharan Africa,30.67,28.79
7,Latin America & Caribbean,51.03,46.16


## Country-Level Detail
1. Which 5 countries saw the largest amount decrease in forest area from 1990 to 2016? What was the difference in forest area for each?
2. Which 5 countries saw the largest percent decrease in forest area from 1990 to 2016? What was the percent change to 2 decimal places for each?
3. If countries were grouped by percent forestation in quartiles, which group had the most countries in it in 2016?
4. List all of the countries that were in the 4th quartile (percent forest > 75%) in 2016.
5. How many countries had a percent forestation higher than the United States in 2016?

### 13. Countries with Largest Increase in Forest Area [1990-2016]

In [14]:
%%read_sql
SELECT f_1990.country_name,
       f_1990.region,
       ROUND((f_2016.forest_area_sqkm - f_1990.forest_area_sqkm)::numeric, 2) AS forest_area_increase
FROM (SELECT country_code, country_name, region, forest_area_sqkm
      FROM forestation
      WHERE year = 1990 AND
            country_code <> 'WLD' AND
            COALESCE(forest_area_sqkm, 0) <> 0) f_1990
JOIN (SELECT country_code, country_name, region, forest_area_sqkm
      FROM forestation
      WHERE year = 2016 AND
            country_code <> 'WLD' AND
            COALESCE(forest_area_sqkm, 0) <> 0) f_2016
ON f_1990.country_code = f_2016.country_code AND
   f_2016.forest_area_sqkm > f_1990.forest_area_sqkm
ORDER BY forest_area_increase DESC
LIMIT 5;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_name,region,forest_area_increase
0,China,East Asia & Pacific,527229.06
1,United States,North America,79200.00
2,India,South Asia,69213.98
3,Russian Federation,Europe & Central Asia,59395.00
4,Vietnam,East Asia & Pacific,55390.00


### 14. Largest Countries in terms of Total Land Area [2016]

In [15]:
%%read_sql
SELECT country_name, ROUND(total_area_sqkm::numeric, 2) AS total_area_sqkm
FROM forestation
WHERE year = 2016 AND
      country_code <> 'WLD' AND
      total_area_sqkm IS NOT NULL
ORDER BY total_area_sqkm DESC
LIMIT 2;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_name,total_area_sqkm
0,Russian Federation,16376870.00
1,China,9388210.01


### 15. Countries with Largest Forest Percentage Increase [1990-2016]

In [16]:
%%read_sql
SELECT f_1990.country_name,
       f_1990.region,
       ROUND(f_1990.total_area_sqkm::numeric, 2) AS total_area_sqkm,
       ROUND((100 * (f_2016.forest_percentage - f_1990.forest_percentage)
             / f_1990.forest_percentage)::numeric, 2) AS forest_pct_change
FROM (SELECT country_code, country_name, region, total_area_sqkm, forest_percentage
      FROM forestation
      WHERE year = 1990 AND
            country_code <> 'WLD' AND
            COALESCE(forest_percentage, 0) <> 0) f_1990
JOIN (SELECT country_code, country_name, region, total_area_sqkm, forest_percentage
      FROM forestation
      WHERE year = 2016 AND
            country_code <> 'WLD' AND
            COALESCE(forest_percentage, 0) <> 0) f_2016
ON f_1990.country_code = f_2016.country_code
ORDER BY forest_pct_change DESC
LIMIT 5;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_name,region,total_area_sqkm,forest_pct_change
0,Iceland,Europe & Central Asia,100249.99,213.66
1,French Polynesia,East Asia & Pacific,3660.01,181.82
2,Bahrain,Middle East & North Africa,690.00,145.91
3,Uruguay,Latin America & Caribbean,175020.00,134.11
4,Dominican Republic,Latin America & Caribbean,48310.00,82.46


### 16. Countries with Largest Forest Area Decrease [1990-2016]

In [17]:
%%read_sql
SELECT f_1990.country_name,
       f_1990.region,
       ROUND(ABS(f_2016.forest_area_sqkm - f_1990.forest_area_sqkm)::numeric
             , 2) forest_amount_delta
FROM (SELECT country_code, country_name, region, forest_area_sqkm
      FROM forestation
      WHERE year = 1990 AND
            country_code <> 'WLD' AND
            COALESCE(forest_area_sqkm, 0) <> 0) f_1990
JOIN (SELECT country_code, country_name, region, forest_area_sqkm
      FROM forestation
      WHERE year = 2016 AND
            country_code <> 'WLD' AND
            COALESCE(forest_area_sqkm, 0) <> 0) f_2016
ON f_1990.country_code = f_2016.country_code
ORDER BY f_2016.forest_area_sqkm - f_1990.forest_area_sqkm
LIMIT 5;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_name,region,forest_amount_delta
0,Brazil,Latin America & Caribbean,541510.00
1,Indonesia,East Asia & Pacific,282193.98
2,Myanmar,East Asia & Pacific,107234.00
3,Nigeria,Sub-Saharan Africa,106506.00
4,Tanzania,Sub-Saharan Africa,102320.00


### 17. Countries with Largest Forest Percentage Decrease [1990-2016]

In [18]:
%%read_sql
SELECT f_1990.country_name,
       f_1990.region,
       ROUND((100 * (f_2016.forest_percentage - f_1990.forest_percentage)
             / f_1990.forest_percentage)::numeric, 2) forest_area_change_pct
FROM (SELECT country_code, country_name, region, forest_percentage
      FROM forestation
      WHERE year = 1990 AND
            country_code <> 'WLD' AND
            COALESCE(forest_percentage, 0) <> 0) f_1990
JOIN (SELECT *
      FROM forestation
      WHERE year = 2016 AND
            country_code <> 'WLD' AND
            COALESCE(forest_percentage, 0) <> 0) f_2016
ON f_1990.country_code = f_2016.country_code
ORDER BY forest_area_change_pct
LIMIT 5;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_name,region,forest_area_change_pct
0,Togo,Sub-Saharan Africa,-75.45
1,Nigeria,Sub-Saharan Africa,-61.80
2,Uganda,Sub-Saharan Africa,-59.27
3,Mauritania,Sub-Saharan Africa,-46.75
4,Honduras,Latin America & Caribbean,-45.03


### 18. Count of Countries Grouped by Forestation Percent Quartiles, 2016

In [19]:
%%read_sql
SELECT CASE WHEN forest_percentage BETWEEN 0 AND 25 THEN 'First Quartile (0-25%)'
            WHEN forest_percentage BETWEEN 25 AND 50 THEN 'Second Quartile (25-50%)'
            WHEN forest_percentage BETWEEN 50 AND 75 THEN 'Third Quartile (50-75%)'
            WHEN forest_percentage BETWEEN 75 AND 100 THEN 'Fourth Quartile (75-100%)'
        END quartile,
       COUNT(1) countries_count
FROM forestation
WHERE year = 2016 AND
      forest_percentage <= 100
GROUP BY 1
ORDER BY 2 DESC;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,quartile,countries_count
0,First Quartile (0-25%),85
1,Second Quartile (25-50%),73
2,Third Quartile (50-75%),38
3,Fourth Quartile (75-100%),9


### 19. Top Quartile Countries, 2016

In [20]:
%%read_sql
SELECT country_code, country_name, region,
       ROUND(forest_percentage::numeric, 2) AS forest_percentage
FROM forestation
WHERE year = 2016 AND
      ROUND(forest_percentage::numeric, 2) > 75
ORDER BY 4 DESC;

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,country_code,country_name,region,forest_percentage
0,SUR,Suriname,Latin America & Caribbean,98.26
1,FSM,"Micronesia, Fed. Sts.",East Asia & Pacific,91.86
2,GAB,Gabon,Sub-Saharan Africa,90.04
3,SYC,Seychelles,Sub-Saharan Africa,88.41
4,PLW,Palau,East Asia & Pacific,87.61
5,ASM,American Samoa,East Asia & Pacific,87.50
6,GUY,Guyana,Latin America & Caribbean,83.90
7,LAO,Lao PDR,East Asia & Pacific,82.11
8,SLB,Solomon Islands,East Asia & Pacific,77.86


### 19. Countries Count Had Higher Forest Percentage than US, 2016

In [21]:
%%read_sql
SELECT COUNT(1)
FROM forestation
WHERE year = 2016 AND
      forest_percentage > (SELECT forest_percentage FROM forestation WHERE country_code = 'USA' AND year = 2016);

Query started at 04:35:56 AM EET; Query executed in 0.00 m

,count
0,94
